# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
from create_tables import *
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

In [2]:
try:
    conn.close()
except:
    pass

In [3]:
# create db and tables
cur, conn = create_database()
    
drop_tables(cur, conn)
create_tables(cur, conn)

conn.close()

OperationalError: database "sparkifydb" is being accessed by other users
DETAIL:  There is 1 other session using the database.


In [ ]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [6]:
def select_star(table, limit=None):
    sql = f"SELECT * FROM {table}"
    if limit:
        sql += f" LIMIT {limit}"
    cur.execute(sql)
    row = cur.fetchone()
    while row:
        print(row)
        row = cur.fetchone()

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [7]:
song_files = get_files('data/song_data/')

In [8]:
len(song_files)

73

In [9]:
filepath = song_files[1]

In [10]:
dfs = [pd.read_json(file, lines=True) for file in song_files]

In [11]:
# df = pd.read_json(filepath, lines=True)
df = pd.concat(dfs)
print(df.shape)
df.reset_index(inplace=True, drop=True)
df.head()

(73, 10)


,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD0S291187B9B7BF5,NaN,Ohio,NaN,Rated R,114.78159,1,SOMJBYD12A6D4F8557,Keepin It Real (Skit),0
1,ARGSJW91187B9B1D6B,35.21962,North Carolina,-80.01955,JennyAnyKind,218.77506,1,SOQHXMF12AB0182363,Young Boy Blues,0
2,AR7G5I41187FB4CE6C,NaN,"London, England",NaN,Adam Ant,233.40363,1,SONHOTT12A8C13493C,Something Girls,1982
3,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0
4,ARKFYS91187B98E58F,NaN,,NaN,Jeff And Sheri Easter,267.70240,1,SOYMRWW12A6D4FAB14,The Moon And I (Ordinary Day Album Version),0


In [12]:
sorted(df.artist_name)

['40 Grit',
 'Adam Ant',
 'Alice Stuart',
 'Andy Andy',
 'Backstreet Boys',
 'Billie Jo Spears',
 'Bitter End',
 'Blingtones',
 'Blue Rodeo',
 'Bob Azzam',
 'Bombay Rockers',
 'Casual',
 'Casual',
 'Casual',
 'Chase & Status',
 'Christos Dantis',
 'Clp',
 'Clp',
 'Danilo Perez',
 'David Martin',
 'Eddie Calvert',
 'Elena',
 'Faiz Ali Faiz',
 'Faye Adams',
 'Five Bolt Main',
 'Glad',
 'Gloriana',
 'Gob',
 'Gwen Stefani',
 'Jeff And Sheri Easter',
 'JennyAnyKind',
 'Jimmy Wakely',
 'Jinx',
 'John Davis',
 'John Wesley',
 'Kenny G featuring Daryl Hall',
 'King Curtis',
 'Line Renaud',
 'Lionel Richie',
 'Los Manolos',
 'Luna Orbit Project',
 'Lupe Fiasco',
 'Marc Shaiman',
 "Mike Jones (Featuring CJ_ Mello & Lil' Bran)",
 'Mitch Ryder',
 'Montserrat Caballé;Placido Domingo;Vicente Sardinero;Judith Blegen;Sherrill Milnes;Georg Solti',
 'Nick Ingman;Gavyn Wright',
 'Planet P Project',
 'Pucho & His Latin Soul Brothers',
 'Rated R',
 'Richard Souther',
 'SUE THOMPSON',
 'Sierra Maestra',
 'S

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [13]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values
tuple(song_data[0])

('SOMJBYD12A6D4F8557',
 'Keepin It Real (Skit)',
 'ARD0S291187B9B7BF5',
 0,
 114.78159)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [14]:
cur.execute(song_table_insert, tuple(song_data[0]))
conn.commit()

In [15]:
# iterate over all the song files
for song_file in tqdm(song_files):
    df_song = pd.read_json(song_file, lines=True)
    # looks like theres only 1 song in each file but just to make this generalizable I'm going
    # to iterate over all rows just in case there's ever more than one song per file
    for i, row in df_song[['song_id', 'title', 'artist_id', 'year', 'duration']].iterrows():
        cur.execute(song_table_insert, row.values)

100%|██████████| 73/73 [00:00<00:00, 104.75it/s]


In [16]:
select_star('songs', 5)

('SOMJBYD12A6D4F8557', 'Keepin It Real (Skit)', 'ARD0S291187B9B7BF5', 0, 114.78159)
('SOQHXMF12AB0182363', 'Young Boy Blues', 'ARGSJW91187B9B1D6B', 0, 218.77506)
('SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', 1982, 233.40363)
('SOUDSGM12AC9618304', 'Insatiable (Instrumental Version)', 'ARNTLGG11E2835DDB9', 0, 266.39628)
('SOYMRWW12A6D4FAB14', 'The Moon And I (Ordinary Day Album Version)', 'ARKFYS91187B98E58F', 0, 267.7024)


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [17]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values
artist_data = tuple(artist_data[0])

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [18]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

In [19]:
# iterate over all the song files extracting artist data
for song_file in tqdm(song_files):
    df_song = pd.read_json(song_file, lines=True)
    # looks like theres only 1 song in each file but just to make this generalizable I'm going
    # to iterate over all rows just in case there's ever more than one song per file
    for i, row in df_song[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].iterrows():
        cur.execute(artist_table_insert, row.values)

100%|██████████| 73/73 [00:00<00:00, 107.35it/s]


In [20]:
select_star('artists',5 )

('ARD0S291187B9B7BF5', 'Rated R', 'Ohio', nan, nan)
('ARGSJW91187B9B1D6B', 'JennyAnyKind', 'North Carolina', 35.21962, -80.01955)
('AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan)
('ARNTLGG11E2835DDB9', 'Clp', '', nan, nan)
('ARKFYS91187B98E58F', 'Jeff And Sheri Easter', '', nan, nan)


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [21]:
log_files = get_files('data/log_data/2018/11/')

In [22]:
len(log_files)

30

In [23]:
filepath = log_files[0]

In [24]:
dfs = [pd.read_json(file, lines=True) for file in log_files]

In [25]:
# df = pd.read_json(filepath, lines=True)
df = pd.concat(dfs)
print(df.shape)
df.head()

(8056, 18)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Dee Dee Bridgewater,Logged In,Lily,F,38,Koch,318.64118,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,La Vie En Rose,200,1542845032796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
1,Tim O'brien,Logged In,Lily,F,39,Koch,176.14322,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Think About Last Night,200,1542845350796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
2,Nirvana,Logged In,Lily,F,40,Koch,215.11791,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Love Buzz,200,1542845526796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
3,Weezer,Logged In,Lily,F,41,Koch,479.32036,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Only In Dreams,200,1542845741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
4,Nightwish,Logged In,Lily,F,42,Koch,286.19710,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,She Is My Sin,200,1542846220796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [26]:
df = df[df.page == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Dee Dee Bridgewater,Logged In,Lily,F,38,Koch,318.64118,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,La Vie En Rose,200,1542845032796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
1,Tim O'brien,Logged In,Lily,F,39,Koch,176.14322,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Think About Last Night,200,1542845350796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
2,Nirvana,Logged In,Lily,F,40,Koch,215.11791,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Love Buzz,200,1542845526796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
3,Weezer,Logged In,Lily,F,41,Koch,479.32036,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Only In Dreams,200,1542845741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
4,Nightwish,Logged In,Lily,F,42,Koch,286.19710,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,She Is My Sin,200,1542846220796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15


In [27]:
df.shape

(6820, 18)

In [28]:
t = pd.to_datetime(df.ts, unit='ms')
t.head()

0   2018-11-22 00:03:52.796
1   2018-11-22 00:09:10.796
2   2018-11-22 00:12:06.796
3   2018-11-22 00:15:41.796
4   2018-11-22 00:23:40.796
Name: ts, dtype: datetime64[ns]

```timestamp, hour, day, week of year, month, year, and weekday```

In [29]:
time_data = (df.ts, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday)
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [30]:
time_df = pd.DataFrame({column_labels[i]: time_data[i] for i in range(len(time_data))})
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,1542845032796,0,22,47,11,2018,3
1,1542845350796,0,22,47,11,2018,3
2,1542845526796,0,22,47,11,2018,3
3,1542845741796,0,22,47,11,2018,3
4,1542846220796,0,22,47,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

```INSERT INTO time (
                                            start_time, 
                                            hour, 
                                            day, 
                                            week, 
                                            month, 
                                            year, 
                                            weekday
                                         ) 
                                      VALUES 
                                         (%s,%s,%s,%s,%s,%s,%s)```

In [31]:
for i, row in tqdm(time_df.iterrows(), total=time_df.shape[0]):
    cur.execute(time_table_insert, list(row))
    conn.commit()

100%|██████████| 6820/6820 [00:10<00:00, 680.03it/s]


In [32]:
select_star('time', 5)

(1542845032796, 0, 22, 47, 11, 2018, 3)
(1542845350796, 0, 22, 47, 11, 2018, 3)
(1542845526796, 0, 22, 47, 11, 2018, 3)
(1542845741796, 0, 22, 47, 11, 2018, 3)
(1542846220796, 0, 22, 47, 11, 2018, 3)


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [33]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [34]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [35]:
for i, row in tqdm(user_df.drop_duplicates().iterrows()):
    #print(list(row))
    cur.execute(user_table_insert, row)
    conn.commit()

137it [00:00, 761.00it/s]


In [36]:
select_star('users', 5)

(15, 'Lily', 'Koch', 'F', 'paid')
(26, 'Ryan', 'Smith', 'M', 'free')
(50, 'Ava', 'Robinson', 'F', 'free')
(97, 'Kate', 'Harrell', 'F', 'paid')
(101, 'Jayden', 'Fox', 'M', 'free')


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [37]:
df[['artist', 'song', 'length']].head()

,artist,song,length
0,Dee Dee Bridgewater,La Vie En Rose,318.64118
1,Tim O'brien,Think About Last Night,176.14322
2,Nirvana,Love Buzz,215.11791
3,Weezer,Only In Dreams,479.32036
4,Nightwish,She Is My Sin,286.19710


In [38]:
saved_results = None
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
#     try:
        artist = row.artist.replace("'", "''")
        cur.execute(song_select,
                    (row.song, row.artist, row.length))
        results = cur.fetchone()
        if results:
            saved_results = results
            print(results, row.values)
#     except Exception as e:
#         print(e)
#         conn.rollback()

 27%|██▋       | 1823/6820 [00:01<00:03, 1663.06it/s]

('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4') ['Elena' 'Logged In' 'Lily' 'F' 5 'Koch' 269.58322 'paid'
 'Chicago-Naperville-Elgin, IL-IN-WI' 'PUT' 'NextSong' 1541048010796.0 818
 'Setanta matins' 200 1542837407796
 '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"'
 '15']


100%|██████████| 6820/6820 [00:04<00:00, 1675.18it/s]


In [41]:
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    artist = row.artist.replace("'", "''")
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, artist, row.length) )
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
        print(songid, artistid, row)
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.itemInSession, row.ts, row.userId, row.level, songid, artistid,
                     row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

 25%|██▌       | 1729/6820 [00:01<00:05, 899.52it/s]

SOZCTXZ12AB0182364 AR5KOSW1187FB35FF4 artist                                                       Elena
auth                                                     Logged In
firstName                                                     Lily
gender                                                           F
itemInSession                                                    5
lastName                                                      Koch
length                                                     269.583
level                                                         paid
location                        Chicago-Naperville-Elgin, IL-IN-WI
method                                                         PUT
page                                                      NextSong
registration                                           1.54105e+12
sessionId                                                      818
song                                                Setanta matins
status                  

100%|██████████| 6820/6820 [00:07<00:00, 913.29it/s]


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [42]:
for query in drop_table_queries:
    cur.execute(query)

In [43]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.